In [ ]:
%pip install ISLP

In [42]:
import numpy as np
import pandas as pd
from matplotlib.pyplot import subplots
import sklearn.model_selection as skm
from ISLP import load_data, confusion_table
from ISLP.models import ModelSpec as MS
import pandas as pd


from sklearn.tree import (DecisionTreeClassifier as DTC,
                          DecisionTreeRegressor as DTR,
                          plot_tree,
                          export_text)
from sklearn.metrics import (accuracy_score,
                             log_loss)
from sklearn.ensemble import \
     (RandomForestRegressor as RF,
      GradientBoostingRegressor as GBR)
from ISLP.bart import BART

In [ ]:
# normalize function 
def normalize_single (x_array, lower, upper):
    min_x = x_array.min()*np.ones(len(x_array))
    max_x = x_array.max()*np.ones(len(x_array))
    lower_bound = lower * np.ones(len(x_array))
    upper_bound = upper * np.ones(len(x_array))
    x_array = lower_bound + (x_array - min_x) * (upper_bound - lower_bound) / (max_x - min_x)
    return x_array

In [43]:
# load the preprocessed training data test.csv
preprocessed_data = pd.read_csv('train_processed.csv')

model = MS(preprocessed_data.columns.drop('Price'), intercept=False)
D = model.fit_transform(preprocessed_data)
feature_names = list(D.columns)
X = np.asarray(D)

# Drop the ID column since it is not a feature
X = np.delete(X, 0, axis=1)

# split the data into training data and testing data
(X_train,
 X_test,
 y_train,
 y_test) = skm.train_test_split(X,
                                preprocessed_data['Price'],
                                test_size=0.3,
                                random_state=0)

print(X_train.shape) # for checking the dimension
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

print(X_train)

(3035, 20)
(1301, 20)
(3035,)
(1301,)
[[1.3095238095238095 0.5097761033405648 1.2364341085271318 ... 0 1 0]
 [1.3095238095238095 0.5153587117445705 1.0629099582587953 ... 1 0 0]
 [1.1666666666666663 0.5112047563097429 1.176505664877758 ... 0 1 0]
 ...
 [1.3095238095238095 0.5068384875971352 1.118067978533095 ... 1 0 0]
 [1.119047619047619 0.5130374199075083 1.107036374478235 ... 0 0 1]
 [1.0714285714285714 0.5167741335964376 1.0217650566487777 ... 0 1 0]]


In [44]:
# Fit a regression tree to the training data
reg = DTR(max_depth=10)
reg.fit(X_train, y_train)
# ax = subplots(figsize=(12,12))[1]
# plot_tree(reg,
#           feature_names=feature_names,
#           ax=ax);

,criterion,'squared_error'
,splitter,'best'
,max_depth,10
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,None
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,ccp_alpha,0.0


In [45]:
# evaluate  Mean Absolute Percentage Error (MAPE) on test data
y_pred = reg.predict(X_test)
mape = np.mean(np.abs((y_test - y_pred) / y_test))
print(f'Mean Absolute Percentage Error (MAPE): {mape:.2f}%')

Mean Absolute Percentage Error (MAPE): 0.25%


In [ ]:
# # cross validation to find the optimal tree depth
# ccp_path = reg.cost_complexity_pruning_path(X_train, y_train)
# kfold = skm.KFold(5,
#                   shuffle=True,
#                   random_state=10)
# grid = skm.GridSearchCV(reg,
#                         {'ccp_alpha': ccp_path.ccp_alphas},
#                         refit=True,
#                         cv=kfold,
#                         scoring='neg_mean_squared_error')
# G = grid.fit(X_train, y_train)

KeyboardInterrupt: 

In [48]:
# Generate a prediction with the test_processed.csv data
test_processed = pd.read_csv('test_processed.csv')
X_test = np.asarray(model.transform(test_processed))
ID_column = X_test[:, 0]  # Save the ID column
X_test = np.delete(X_test, 0, axis=1)  # Remove the ID column for prediction
print(X_test.shape)
# just take 1491 rows for submission
X_test = X_test[0:1491, :]
print(X_test.shape)
y_pred = reg.predict(X_test)
output = pd.DataFrame({'ID': ID_column, 'Price': y_pred})
print(output.shape)
output.to_csv('submission.csv', index=False)

(1491, 20)
(1491, 20)


ValueError: could not convert string to float: '17.8 kmpl'

In [ ]:
# best_ = grid.best_estimator_

# # evaluate  Mean Absolute Percentage Error (MAPE) on test data
# y_pred = best_.predict(X_test)
# mape = np.mean(np.abs((y_test - y_pred) / y_test))
# print(f'Mean Absolute Percentage Error (MAPE) after pruning: {mape:.2f}%')

Mean Absolute Percentage Error (MAPE) after pruning: 0.31%
